# 2.1
a) Function constructing VX and VY

In [1]:
import numpy as np
def xy(x0, y0, L1, L2, noelms1, noelms2):
    lx = L1 / noelms1
    ly = L2 / noelms2

    VX = np.repeat([x0 + i * lx for i in range(noelms1 + 1)], noelms2 + 1)
    VY = [y0 + j * ly for j in range(noelms2,-1,-1)] * (noelms1 + 1)

    return VX, VY

In [5]:
# x0 = 0
# y0 = 0
(x0, y0) = (-2.5,-4.8)
L1 = 7.6
L2 = 5.9
noelms1 = 4
noelms2 = 3


VX,VY = xy(x0,y0,L1,L2,noelms1,noelms2)

print(VX)
print(VY)

print(len(VX))
print(len(VY))


[-2.5 -2.5 -2.5 -2.5 -0.6 -0.6 -0.6 -0.6  1.3  1.3  1.3  1.3  3.2  3.2
  3.2  3.2  5.1  5.1  5.1  5.1]
[1.1000000000000005, -0.8666666666666663, -2.833333333333333, -4.8, 1.1000000000000005, -0.8666666666666663, -2.833333333333333, -4.8, 1.1000000000000005, -0.8666666666666663, -2.833333333333333, -4.8, 1.1000000000000005, -0.8666666666666663, -2.833333333333333, -4.8, 1.1000000000000005, -0.8666666666666663, -2.833333333333333, -4.8]
20
20


b) Function constructing EToV

In [6]:
def conelmtab(noelms1,noelms2):
    EToV = []
    for j in range(noelms1):
        for i in range(noelms2):
            EToV.append([i+noelms2+1+(noelms2+1)*j,i+(noelms2+1)*j, i+noelms2+2+(noelms2+1)*j])
            EToV.append([i+1+(noelms2+1)*j,i+noelms2+2+(noelms2+1)*j,i+(noelms2+1)*j])

    return np.array(EToV)

EToV = conelmtab(4,3)

# 2.2

a) basis function.

In [7]:
from matplotlib import pyplot as plt

def basfun(n,VX,VY,EToV):
    idx1 = EToV[n,0]
    idx2 = EToV[n,1]    
    idx3 = EToV[n,2]

    x1 = VX[idx1]
    x2 = VX[idx2]
    x3 = VX[idx3]

    y1 = VY[idx1]
    y2 = VY[idx2]
    y3 = VY[idx3]

    delta = 1/2*(x2*y3 - x3*y2 - x1*y3 + x3*y1 + x1*y2 - x2*y1)

    abc = np.zeros((3,3))

    abc[0,0] = x2*y3 - x3*y2
    abc[0,1] = y2 - y3
    abc[0,2] = x3 - x2

    abc[1,0] = x3*y1 - x1*y3
    abc[1,1] = y3 - y1
    abc[1,2] = x1 - x3

    abc[2,0] = x1*y2 - x2*y1
    abc[2,1] = y1 - y2
    abc[2,2] = x2 - x1


    return delta, abc


    # plt.plot(VX,VY,'o')
    # plt.plot([x1,x2,x3],[y1,y2,y3],'o')

    
    

delta, abc = basfun(3,VX,VY,EToV)

print(delta)
print(abc)

1.8683333333333327
[[-6.56333333 -1.96666667 -1.9       ]
 [ 4.91666667  1.96666667  0.        ]
 [ 5.38333333  0.          1.9       ]]


b) normal vector component function

In [8]:
def outernormal(n,k,VX,VY,EToV):
    idx1 = EToV[n,(k % 3)]
    idx2 = EToV[n,((k + 1) % 3)]

    dx = VX[idx2] - VX[idx1]
    dy = VY[idx2] - VY[idx1]

    n = np.array([-dy,dx])/np.sqrt(dx*dx+dy*dy)

    return n

outernormal(8,1,VX,VY,EToV)

array([0.71919139, 0.69481202])

# 2.3
a) assembly process

In [9]:
import scipy.sparse as sp

def assembly(VX, VY, EToV, lam1,lam2, qt):
    N = len(EToV[:,1])
    M = len(VX)

    nnzmax = 9*N
    ii = np.zeros(nnzmax, dtype=int)
    jj = np.zeros(nnzmax, dtype=int)
    ss = np.zeros(nnzmax)
    b = np.zeros(M)
    count = 0
    
    for n in range(N):
        delta, abc = basfun(n,VX,VY,EToV)

        i = EToV[n,0]
        j = EToV[n,1]    
        k = EToV[n,2]
            
        triplet = [i,j,k]

        x1 = VX[i]
        x2 = VX[j]
        x3 = VX[k]

        y1 = VY[i]
        y2 = VY[j]
        y3 = VY[k]
        
        for r in range(3):
        
            qtilde = 1/3*(qt(x1,y1)+qt(x2,y2)+qt(x3,y3))
            b[triplet[r]] += (np.abs(delta)/3)*qtilde

            for s in range(3):
                
                k_rs = 1/(4*np.abs(delta))*(lam1*abc[r,1]*abc[s,1]+lam2*abc[r,2]*abc[s,2])
                
                ii[count] = triplet[r]
                jj[count] = triplet[s]
                ss[count] = k_rs
                count += 1
        
    A = sp.csr_matrix((ss[:count], (ii[:count], jj[:count])), shape=(M, M))

    return A,b

In [21]:
(x0, y0) = (0,0) 
L1 = 1
L2 = 1
noelms1 = 4
noelms2 = 4
q = lambda x,y: 0
lam1 = 1
lam2 = 1

# (x0, y0) = (-2.5, -4.8) 
# L1 = 7.6
# L2 = 5.9
# noelms1 = 4
# noelms2 = 3
# q = lambda x,y: -6*x+2*y-2
# lam1 = 1
# lam2 = 1

VX,VY = xy(x0,y0,L1,L2,noelms1,noelms2)
EToV = conelmtab(noelms1,noelms2)


A,b = assembly(VX, VY, EToV, lam1,lam2, q)


In [23]:
np.diag(A.todense(), k=-1)

array([-0.48305085, -0.48305085, -0.48305085,  0.        , -0.96610169,
       -0.96610169, -0.96610169,  0.        , -0.96610169, -0.96610169,
       -0.96610169,  0.        , -0.96610169, -0.96610169, -0.96610169,
        0.        , -0.48305085, -0.48305085, -0.48305085])

# 2.4

In [9]:
def dirbc(bnodes, f, A, b):
    M = len(b)
    for n,i in enumerate(bnodes):
        A[i,i] = 1
        b[i] = f[n]

        for j in range(M):
            if j != i:
                A[i,j] = 0
                if j not in bnodes:
                    b[j] -= A[j,i]*f[n]
                    A[j,i] = 0
    
    return A,b

In [10]:
def find_bnodes(noelms1,noelms2):
    for i in range(noelms1+1):
        bnodes = np.arange(noelms1)
        for j in range(1,noelms2+1):
            bnodes = np.append(bnodes, j*(noelms1))
            bnodes = np.append(bnodes, j*(noelms1) + noelms2-1)

        bnodes = np.append(bnodes, np.arange((noelms2+1)*noelms1, (noelms2+1)*(noelms1+1)))

    return bnodes

In [20]:
(x0, y0) = (0,0) 
L1 = 1
L2 = 1
noelms1 = 4
noelms2 = 4
q = lambda x,y: 0
lam1 = 1
lam2 = 1
bnodes = find_bnodes(noelms1,noelms2)
f = np.ones((noelms1+1)*(noelms2+1))


VX,VY = xy(x0,y0,L1,L2,noelms1,noelms2)
EToV = conelmtab(noelms1,noelms2)
A,b = assembly(VX, VY, EToV, lam1,lam2, q)

# A, b = dirbc(bnodes, f, A, b)

print(A.todense())

NameError: name 'find_bnodes' is not defined

In [13]:
from GlobalAssembly import assembly as assembly2

(x0, y0) = (0,0) 
L1 = 1
L2 = 1
noelms1 = 4
noelms2 = 4
q = lambda x,y: 0
lam1 = 1
lam2 = 1

VX,VY = xy(x0,y0,L1,L2,noelms1,noelms2)
EToV = conelmtab(noelms1,noelms2)

A,b = assembly2(VX, VY, EToV, lam1,lam2, q)

print(A.diagonal())


[1.   2.   2.   2.   3.25 4.5  4.   4.   4.   6.75 6.75 4.   4.   4.
 6.75 6.75 3.   2.   2.   3.5  3.25 1.   0.   0.   0.  ]
